# Using TensorFlow to run Monte Carlo Simulations

This notebook will show how I worked through creating a Monte Carlo Simulation using TensorFlow. The inspiration for this post comes from [Matthias Groncki](https://ipythonquant.wordpress.com/2018/05/22/tensorflow-meets-quantitative-finance-pricing-exotic-options-with-monte-carlo-simulations-in-tensorflow/). Matthias covered multiple options but I will only cover the plain vanilla call at first since that is what I use to trade (poorly I might add).

**Current Environment**  
Windows 10  
Python 3.6.1  
Anaconda3 4.4.0  
TensorFlow 1.2.0  
Keras 2.0.5  
NumPy 1.12.1  
MatPlotLib 2.0.2  